In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import folium
from folium.plugins.marker_cluster import MarkerCluster
import json

In [ ]:
# Revoir le chemin d'accès, je l'ai remis dans le dossier local slcities, pas le GH
# le lien du site pour convertir le shp en (geo)json : 
# https://www.statsilk.com/maps/convert-esri-shapefile-map-geojson-format

# GeoDataFrame des contours IRIS France Metropolitaine

## Tri des valeurs pour Nice

In [116]:
nice_gpd_raw = gpd.read_file('../raw_data/gpd_nice.json')

In [117]:
nice_gpd = nice_gpd_raw.sort_values(by='NOM_IRIS')

In [118]:
nice_gpd['CODE_IRIS']= nice_gpd['CODE_IRIS'].astype('int64')

## Création de la colonne area

In [119]:
##### calcule l'aire de chaque quartier, en m²
nice_gpd['area']= nice_gpd.area

/tmp/ipykernel_580/1154274972.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  nice_gpd['area']= nice_gpd.area


In [120]:
# vérification que la surface totale ~ 71km²
# (la découpe des limites n'est pas très précise, d'où le delta de 2)

nice_gpd.area.sum()

/tmp/ipykernel_580/1759359854.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  nice_gpd.area.sum()


73691407.30671854

In [121]:
# on met ça en km²
nice_gpd['area'] = nice_gpd['area'].map(lambda x : round(x/10**6, 2))

In [122]:
nice_gpd['area'].sum()

73.68

## Conversion des polygones de la projection Lambert à la norme WSG84

In [32]:
nice_gpd.crs={'init':'epsg:2154'}

/tmp/ipykernel_580/1866397672.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  nice_gpd.crs={'init':'epsg:2154'}


In [33]:
# la norme WSG84 correspond à 4236
nice_gpd = nice_gpd.to_crs(epsg=4326)

AttributeError: 'DataFrame' object has no attribute 'to_crs'

In [499]:
#les polygones sont bien dans le format géodésique classique
nice_gpd.head()

,INSEE_COM,NOM_COM,IRIS,CODE_IRIS,NOM_IRIS,TYP_IRIS,geometry,area
26041,06088,Nice,2705,60882705,Abbaye de Roseland,H,"POLYGON ((7.23201 43.69345, 7.23197 43.69347, ...",0.38
33084,06088,Nice,1602,60881602,Acropolis-XVe Corps,H,"POLYGON ((7.27902 43.71052, 7.27866 43.71056, ...",0.26
40380,06088,Nice,3001,60883001,Arenas-Aéroport,A,"POLYGON ((7.20012 43.66450, 7.19960 43.66430, ...",4.30
35097,06088,Nice,3002,60883002,Arenas-Cassin,H,"POLYGON ((7.21943 43.67236, 7.21937 43.67232, ...",0.16
6210,06088,Nice,0901,60880901,Ariane-Les Chênes,H,"POLYGON ((7.29541 43.73765, 7.29538 43.73760, ...",0.15


# DataFrame recensement Nice

In [500]:
nice_nb_hab = pd.read_csv('../raw_data/recensement_nice.csv')

In [501]:
nice_nb_hab.tail()

,Territoire,Code,Population / Homme,Population / Femme,Population / Total
142,Vieux Nice-Sainte Reparate,60880102,817,948,1766
143,Vieux Nice-Visitation,60880103,868,1042,1911
144,Villa Arson,60881305,1120,1271,2391
145,Vinaigrier,60880801,853,805,1658
146,Nice,6088,160407,183220,343631


In [502]:
# Je vire le total de la ville
nice_hab = nice_nb_hab.drop(146).rename(columns={'Code':'CODE_IRIS'})

In [503]:
nice_hab.tail()

,Territoire,CODE_IRIS,Population / Homme,Population / Femme,Population / Total
141,Victor Hugo-Buffa,60882402,1093,1416,2509
142,Vieux Nice-Sainte Reparate,60880102,817,948,1766
143,Vieux Nice-Visitation,60880103,868,1042,1911
144,Villa Arson,60881305,1120,1271,2391
145,Vinaigrier,60880801,853,805,1658


# Combinaison des deux DF

In [504]:
df_merge_nice = pd.merge(nice_gpd, nice_hab, how='inner', on='CODE_IRIS')

In [505]:
geodata_nice = df_merge_nice[['CODE_IRIS','NOM_IRIS','geometry','area','Population / Total']].rename(columns={'Population / Total':'population'})

In [506]:
geodata_nice.head()

,CODE_IRIS,NOM_IRIS,geometry,area,population
0,60882705,Abbaye de Roseland,"POLYGON ((7.23201 43.69345, 7.23197 43.69347, ...",0.38,2019
1,60881602,Acropolis-XVe Corps,"POLYGON ((7.27902 43.71052, 7.27866 43.71056, ...",0.26,2256
2,60883001,Arenas-Aéroport,"POLYGON ((7.20012 43.66450, 7.19960 43.66430, ...",4.30,580
3,60883002,Arenas-Cassin,"POLYGON ((7.21943 43.67236, 7.21937 43.67232, ...",0.16,2871
4,60880901,Ariane-Les Chênes,"POLYGON ((7.29541 43.73765, 7.29538 43.73760, ...",0.15,2375


In [507]:
# Check population totale
geodata_nice['population'].sum()

343631

In [508]:
geodata_nice['density'] = round(geodata_nice['population']/geodata_nice['area'],2)

In [509]:
#Vérification de la moyenne de densité
geodata_nice['population'].sum()/geodata_nice['area'].sum()

4663.830076004343

In [510]:
# Comparé aux chiffres mesurés
342522/71.92

4762.541713014461

# Export des DF

In [115]:
# Pour conserver un DF bien plus léger des contours
with open("../raw_data/gpd_nice.json", "w") as f:
    f.write(nice_gpd_raw1.to_json())

In [514]:
# DF du recensement retravaillé
nice_hab.to_csv('../raw_data/habitants_nice_cleaned.csv',index=False)

In [511]:
# Sauvegarde du GPD nettoyé sur les données des quartiers
with open("../raw_data/gpd_nice_cleaned.json", "w") as f:
    f.write(nice_gpd.to_json())

In [485]:
# Sauvegarde du GPD combiné
with open("../slcities/data/geodata_nice.json", "w") as f:
    f.write(geodata_nice.to_json())

# Carte de Nice

In [93]:
#geodata_nice = gpd.read_file('../slcities/data/geodata_nice.json')

In [106]:
m = folium.Map(location=[43.7031, 7.2661], zoom_start=12,tiles="Stamen Toner")

density_layer = folium.Choropleth(
    geo_data=geodata_nice,
    name="densité",
    data=geodata_nice,
    columns=['NOM_IRIS','density'],
    key_on="feature.properties.NOM_IRIS",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name="density",
    highlight=True,
).add_to(m)


##############  10 points de style   ############## 

# style_function = lambda x: {'fillColor': '#ffffff', 
#                            'color':'#000000', 
#                            'fillOpacity': 0.1, 
#                            'weight': 0.1}
# highlight_function = lambda x: {'fillColor': '#000000', 
#                                'color':'#000000', 
#                                'fillOpacity': 0.50, 
#                                'weight': 0.1}
# NIL = folium.features.GeoJson(
#    data = geodata_nice,
#    style_function=style_function, 
#    control=False,
#    highlight_function=highlight_function, 
#    tooltip=folium.features.GeoJsonTooltip(
#        fields=['NOM_IRIS','density'],
#        aliases=['Nom du quartier','densité de population'],
#        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
#    )
# )
# m.add_child(NIL)
# m.keep_in_front(NIL)

m